In [65]:
#optional code if warnings become over bearing 
import warnings
warnings.filterwarnings('ignore')

In [126]:
%matplotlib inline

import os
import pickle
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import date
from datetime import datetime
from datetime import timedelta

from darksky import forecast

import config

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import TwoByTwoLayout
from ipywidgets import GridspecLayout
import threading
from IPython.display import display, HTML
import ipywidgets as widgets
import time

RAPIDAPI_KEY  = config.darksky_api1

## Loading Models & Data

In [67]:
df_fires = pd.read_csv('data/FireIntensity_Model_June12_Clean.csv')

## Defining widgets

In [68]:
datepicker = widgets.DatePicker(
    description='Fire start: ',
    disabled=False,
    value = date.today()
)

In [69]:
temperature = widgets.IntSlider(value=85, max=120, min=20, description='Temperature: ')
temperature.style.handle_color = 'lightblue'

In [70]:
humidity = widgets.IntSlider(value=85, max=120, min=20, description='Humidity: ')

In [71]:
lat = widgets.FloatSlider(value=df_fires.latitude.median(),
                          max=df_fires.latitude.max(),
                          min=df_fires.latitude.min(),
                          description='Latitude: ',
                          readout_format='.4f',
                          step=0.0001,
                         )
lat.style.handle_color = 'lightpink'

lon = widgets.FloatSlider(value=df_fires.longitude.median(),
                          max=df_fires.longitude.max(),
                          min=df_fires.longitude.min(),
                          description='Longitude: ',
                          readout_format='.4f',
                          step=0.0001,
                         )
lon.style.handle_color = 'lightpink'

In [72]:
fuel_moist = widgets.Dropdown(
    options=[('Very Dry', 1), ('Dry', 2), ('Moderate', 3), ('Moist', 4)],
    value=1,
    description='Fuel Moisture Class:',
)

In [73]:
prefire = widgets.FloatSlider(value=df_fires.prefire_fuel.median(),
                          max=df_fires.prefire_fuel.max(),
                          min=df_fires.prefire_fuel.min(),
                          description='Prefire Fuel: ',
                          readout_format='.5f',
                          step=0.00001,
                         )
prefire.style.handle_color = 'lightblue'

In [98]:
fuel_code = widgets.Dropdown(
    options=[('Herbaceous ', 1), 
             ('Shrub / scrub ', 2),
             ('Spruce / fir group ', 1120), 
             ('Longleaf / slash pine group ', 1140), 
             ('Loblolly / shortleaf pine group ', 1160), 
             ('Douglas-fir group ', 1200), 
             ('Hemlock / Sitka spruce group ', 1300), 
             ('Redwood group ', 1340), 
             ('Oak / hickory group ', 1500), 
             ('Elm / ash / cottonwood group ', 1700)],
    value=1,
    description='Cover Type:',
)

## Predicting Fire Intensity App

#### Enter the information in app below and submit for pridiction on fire intensity

In [150]:
fireapp = GridspecLayout(4, 3)
fireapp[0,0] = datepicker
fireapp[1,0] = lat
fireapp[2,0] = lon
fireapp[0,1] = fuel_moist
fireapp[1,1] = fuel_code
fireapp[2,1] = prefire
fireapp[0,2] = button

In [145]:
#fireapp

In [102]:
#source: https://towardsdatascience.com/heres-how-to-calculate-distance-between-2-geolocations-in-python-93ecab5bbba4

# estimate fire region input
def haversine_distance(lat1, lon1, lat2, lon2):
    r = 6371
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2
    res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
    return np.round(res, 2)

distances_km = []

for row in df_fires.itertuples(index=False):
    distances_km.append(
       haversine_distance(lat.value, lon.value, row.latitude, row.longitude)
   )

df_fires['DistanceFromPoint'] = distances_km

fire_region_in = df_fires[df_fires.DistanceFromPoint == df_fires.DistanceFromPoint.min()].fire_region.min()

In [103]:
#get doy input
def get_season(doy):
    if ((doy >= 80) and (doy <= 172)):
        s = 0  # spring
    elif ((doy > 172) and (doy < 264)):
        s = 1  # summer
    elif ((doy >= 264) and (doy <= 355)):
        s = 2  # fall
    #elif ((doy > 355) and (doy < 80)):
        #s = 3  # winter
    else:
        s = 3
     #   raise IndexError("Invalid date")
    return s

season_in = get_season(datepicker.value.timetuple().tm_yday)

In [104]:
def weather_lookup(df, key=RAPIDAPI_KEY, days_before=0, days_after=0):
    data = []
    for index, row in df.iterrows():
        ts = row['datetime'].isoformat() 
        lat = row['latitude']
        lon = row['longitude']
        weather = forecast(key, lat, lon, time=ts)
        w_dict = weather['currently']
        w_dict['timestamp'] = ts
        w_dict['latitude'] = lat
        w_dict['longitude'] = lon
        data.append(w_dict)
    return data


In [106]:
dt = datetime(datepicker.value.year, datepicker.value.month, datepicker.value.day, 12)
fire_predict = pd.DataFrame(columns=['datetime', 'latitude', 'longitude'])
fire_predict = fire_predict.append({'datetime': dt, 'latitude': lat.value, 'longitude': lon.value}, ignore_index=True)
weather_df = pd.DataFrame(weather_lookup(fire_predict))

temperature_in = weather_df.temperature.min()
wind_speed_in = weather_df.windSpeed.min()
humidity_in = weather_df.humidity.min()
precip_intensity_in = weather_df.precipIntensity.min()
wind_gust_in = weather_df.windGust.min()

In [112]:
print("------User Inputs------")
print("Fire Start: ", datepicker.value)
print("Latitude: ", lat.value)
print("Longitude: ", lon.value)
print("Fuel Moisture Class: ", fuel_moist.value)
print("Prefire Fuel: ", prefire.value)
print("Fuelcode: ", fuel_code.value)
print("")


print("------Model Input Values------")
print("Latitude: ", lat.value)
print("Longitude: ", lon.value)
print("Day of Year: ", datepicker.value.timetuple().tm_yday)
print("Fuelcode: ", fuel_code.value)
print("Fuel Moisture Class: ", fuel_moist.value)
print("Prefire Fuel: ", prefire.value)
print("Temperature: ", temperature_in)
print("Humidity: ", humidity_in)
print("Precip Intensity: ", precip_intensity_in)
print("Wind Gust: ", wind_gust_in)
print("Wind Speed: ", wind_speed_in)
print("")

print("----Additional Features-----")
print("Season: ", season_in)
print("Fire Region: ", fire_region_in)


------User Inputs------
Fire Start:  2008-06-18
Latitude:  38.8183
Longitude:  -109.0128
Fuel Moisture Class:  1
Prefire Fuel:  6971.76675
Fuelcode:  1500

------Model Input Values------
Latitude:  38.8183
Longitude:  -109.0128
Day of Year:  170
Fuelcode:  1500
Fuel Moisture Class:  1
Prefire Fuel:  6971.76675
Temperature:  82.36
Humidity:  0.15
Precip Intensity:  0
Wind Gust:  13.11
Wind Speed:  5.64

----Additional Features-----
Season:  0
Fire Region:  4


In [ ]:
features = ['latitude', 'longitude', 'doy', 'fuelcode', 'fuel_moisture_class', 'prefire_fuel', 'temperature', 'humidity', 'precip_intensity', 'wind_gust', 'wind_speed']

In [381]:
# pull from darksky api
'temperature', 'wind_speed','humidity','visibility', 'precip_intensity', 'wind_gust'

#use functions to create
'fire_region', 'season',

#set by user
'fuel_moisture_class','fuelcode', 'prefire_fuel'



('fuel_moisture_class', 'fuelcode', 'prefire_fuel')

In [307]:
## Other cool stuff to try

In [79]:

a = widgets.IntSlider(description='a')
a.style.handle_color = 'lightblue'
b = widgets.IntSlider(description='b')
b.style.handle_color = 'green'
c = widgets.IntSlider(description='c')
c.style.handle_color = 'red'
def f(a, b, c):
    print('{}*{}*{}={}'.format(a, b, c, a*b*c))

out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

widgets.HBox([widgets.VBox([a, b, c]), out])


In [80]:

widgets.ToggleButtons(
    options=['Slow', 'Regular', 'Fast'],
    description='Speed:',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['Description of slow', 'Description of regular', 'Description of fast'],
#     icons=['check'] * 3
)

ToggleButtons(description='Speed:', options=('Slow', 'Regular', 'Fast'), tooltips=('Description of slow', 'Des…

In [81]:

accordion = widgets.Accordion(children=[widgets.IntSlider(), widgets.Text("test")], titles=('Slider', 'Text'))
accordion

Accordion(children=(IntSlider(value=0), Text(value='test')))

In [182]:
from IPython.display import display
button = widgets.Button(description="Predict Fire")
output = widgets.Output()

display(button, output)

def on_button_clicked(b):
    with output:
        print("Warming up...")
#         print("Finding region on map...")
#         def haversine_distance(lat1, lon1, lat2, lon2):
#             r = 6371
#             phi1 = np.radians(lat1)
#             phi2 = np.radians(lat2)
#             delta_phi = np.radians(lat2 - lat1)
#             delta_lambda = np.radians(lon2 - lon1)
#             a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2
#             res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
#             return np.round(res, 2)

#         distances_km = []

#         for row in df_fires.itertuples(index=False):
#             distances_km.append(
#             haversine_distance(lat.value, lon.value, row.latitude, row.longitude)
#          )

#         df_fires['DistanceFromPoint'] = distances_km

#         fire_region_in = df_fires[df_fires.DistanceFromPoint == df_fires.DistanceFromPoint.min()].fire_region.min()
#         print('### remove this ###  Region: ',fire_region_in)
        
        #get season input
        def get_season(doy):
            if ((doy >= 80) and (doy <= 172)):
                s = 0  # spring
            elif ((doy > 172) and (doy < 264)):
                s = 1  # summer
            elif ((doy >= 264) and (doy <= 355)):
                s = 2  # fall
            #elif ((doy > 355) and (doy < 80)):
                #s = 3  # winter
            else:
                s = 3
             #   raise IndexError("Invalid date")
            return s
        
        season_in = get_season(datepicker.value.timetuple().tm_yday)
        print('### remove this ###  Season: ',season_in)
        
        
        print("Pulling weather data from DarkSky...")
        def weather_lookup(df, key=RAPIDAPI_KEY, days_before=0, days_after=0):
            data = []
            for index, row in df.iterrows():
                ts = row['datetime'].isoformat() 
                lat = row['latitude']
                lon = row['longitude']
                weather = forecast(key, lat, lon, time=ts)
                w_dict = weather['currently']
                w_dict['timestamp'] = ts
                w_dict['latitude'] = lat
                w_dict['longitude'] = lon
                data.append(w_dict)
            return data
        
        dt = datetime(datepicker.value.year, datepicker.value.month, datepicker.value.day, 12)
        fire_predict = pd.DataFrame(columns=['datetime', 'latitude', 'longitude'])
        fire_predict = fire_predict.append({'datetime': dt, 'latitude': lat.value, 'longitude': lon.value}, ignore_index=True)
        weather_df = pd.DataFrame(weather_lookup(fire_predict))
        
        temperature_in = weather_df.temperature.min()
        wind_speed_in = weather_df.windSpeed.min()
        humidity_in = weather_df.humidity.min()
        precip_intensity_in = weather_df.precipIntensity.min()
        wind_gust_in = weather_df.windGust.min()
        
        print('### remove this ###  Weather Data: ')
        print("Temperature: ", temperature_in)
        print("Humidity: ", humidity_in)
        print("Precip Intensity: ", precip_intensity_in)
        print("Wind Gust: ", wind_gust_in)
        print("Wind Speed: ", wind_speed_in)
        
        print("Calculating Intinsity...")
        def thread_func(something, out):
            for i in range(1, 5):
                time.sleep(2.5)
                out.append_stdout('{} {} {}\n'.format(i, '**'*i, something))
            #out.append_display_data(HTML("<em>All done!</em>"))
            print("Fire is SEVERE!")
        
        display('Display in main thread')
        out = widgets.Output()
        # Now the key: the container is displayed (while empty) in the main thread
        display(out)
        
        thread = threading.Thread(
            target=thread_func,
            args=("Calculating", out))
        thread.start()
        
button.on_click(on_button_clicked)

Button(description='Predict Fire', style=ButtonStyle())

Output()

In [189]:
display(fireapp, output)

GridspecLayout(children=(DatePicker(value=datetime.date(2012, 1, 18), description='Fire start: ', layout=Layou…

Output()

In [186]:
output

Output()

In [185]:
output.clear_output()

In [173]:
fireapp = GridspecLayout(4, 3)
fireapp[0,0] = datepicker
fireapp[1,0] = lat
fireapp[2,0] = lon
fireapp[0,1] = fuel_moist
fireapp[1,1] = fuel_code
fireapp[2,1] = prefire
fireapp[0,2] = button

In [159]:
%gui asyncio

In [160]:

import asyncio
def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # make the new value available
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future

In [166]:

from functools import wraps
def yield_for_change(widget, attribute):
    """Pause a generator to wait for a widget change event.

    This is a decorator for a generator function which pauses the generator on yield
    until the given widget attribute changes. The new value of the attribute is
    sent to the generator and is the value of the yield.
    """
    def f(iterator):
        @wraps(iterator)
        def inner():
            i = iterator()
            def next_i(change):
                try:
                    i.send(change.new)
                except StopIteration as e:
                    widget.unobserve(next_i, attribute)
            widget.observe(next_i, attribute)
            # start the generator
            next(i)
        return inner
    return f

In [168]:
from ipywidgets import IntSlider, VBox, HTML
slider2=IntSlider()

@yield_for_change(slider2, 'value')
def f():
    for i in range(10):
        print('did work %s'%i)
        x = yield
        print('generator function continued with value %s'%x)
f()

slider2

did work 0


IntSlider(value=0)

generator function continued with value 45
did work 1
generator function continued with value 19
did work 2
generator function continued with value 75
did work 3
generator function continued with value 55
did work 4
generator function continued with value 26
did work 5
generator function continued with value 92
did work 6
generator function continued with value 39
did work 7
generator function continued with value 9
did work 8
generator function continued with value 49
did work 9
generator function continued with value 89
